In [ ]:
import json

import pandas as pd
from util import mock_snakemake

if "snakemake" not in globals() or hasattr(snakemake, "mock"):  # noqa: F821
    snakemake = mock_snakemake("process_line_data")

In [6]:
df = pd.json_normalize(json.load(open(snakemake.input[0])), record_path=["features"])
df.head()

,id,type,geometry.coordinates,geometry.type,properties.br_r,properties.rate_a,properties.Comments,properties.br_b,properties.Lon2,properties.Lon1,...,properties.Lat1,properties.f_bus,properties.t_bus,properties.CATS_ID,properties.Lat2,properties.transformer,properties.Structure_Type,properties.Type,properties.Circuit,properties.Structure_Material
0,0,Feature,"[[-122.24291409889523, 37.77242586670423], [-1...",LineString,0.000048,1.75,,0.000027,-122.243143,-122.242914,...,37.772426,1,2410,1,37.772041,False,NaN,NaN,NaN,NaN
1,1,Feature,"[[-122.24314284220173, 37.77204138338957], [-1...",LineString,0.000914,4.37,,0.002321,-122.214479,-122.243143,...,37.772041,2410,20,2,37.763002,False,NaN,NaN,NaN,NaN
2,2,Feature,"[[-122.24314284220173, 37.77204138338957], [-1...",LineString,0.003540,1.85,,0.002469,-122.291454,-122.243143,...,37.772041,2410,2411,3,37.779166,False,NaN,NaN,NaN,NaN
3,3,Feature,"[[-122.29145408356094, 37.779165922885475], [-...",LineString,0.000156,1.42,,0.000071,-122.292698,-122.291454,...,37.779166,2411,1618,4,37.779322,False,NaN,NaN,NaN,NaN
4,4,Feature,"[[-122.29145408356094, 37.779165922885475], [-...",LineString,0.000648,2.19,,0.000537,-122.289818,-122.291454,...,37.779166,2411,1617,5,37.788672,False,NaN,NaN,NaN,NaN


In [ ]:
df2 = df.drop(columns=(c for c in df.columns if not c.startswith("properties.")))
df2.columns = df2.columns.str.replace("properties.", "", regex=False)
df2.head()

,br_r,rate_a,Comments,br_b,Lon2,Lon1,br_x,kV,Lat1,f_bus,t_bus,CATS_ID,Lat2,transformer,Structure_Type,Type,Circuit,Structure_Material
0,0.000048,1.750000,,0.000027,-122.243143,-122.242914,0.000219,115.0,37.772426,1,2410,1,37.772041,False,NaN,NaN,NaN,NaN
1,0.000914,4.370000,,0.002321,-122.214479,-122.243143,0.007485,115.0,37.772041,2410,20,2,37.763002,False,NaN,NaN,NaN,NaN
2,0.003540,1.850000,,0.002469,-122.291454,-122.243143,0.016138,115.0,37.772041,2410,2411,3,37.779166,False,NaN,NaN,NaN,NaN
3,0.000156,1.420000,,0.000071,-122.292698,-122.291454,0.000469,115.0,37.779166,2411,1618,4,37.779322,False,NaN,NaN,NaN,NaN
4,0.000648,2.190000,,0.000537,-122.289818,-122.291454,0.004104,115.0,37.779166,2411,1617,5,37.788672,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10818,0.000444,3.820000,,0.000000,-114.142301,-114.142291,0.028531,NaN,34.293298,8724,8726,10745,34.293283,True,,,,
10819,0.018577,0.469016,,0.000000,-116.532562,-116.532521,0.464432,NaN,33.812203,7687,7686,10197,33.812205,True,,,,
10820,0.021783,0.200000,,0.000000,-117.040123,-117.040139,0.544565,NaN,34.088329,7877,7876,10297,34.088271,True,,,,
10821,0.003790,0.836464,,0.000000,-121.760253,-121.760322,0.130255,NaN,36.951969,8693,8692,10727,36.951914,True,,,,


In [ ]:
col_name_map = {
    # "CATS_ID": "id",
    "f_bus": "from_bus",
    "t_bus": "to_bus",
    # "kV": "voltage_kv",
    "rate_a": "line_rating",
    "br_r": "resistance",
}
df3 = df2.rename(columns=col_name_map)
df3 = df3[col_name_map.values()]
df3.head()

,from_bus,to_bus,line_rating,resistance
0,1,2410,1.750000,0.000048
1,2410,20,4.370000,0.000914
2,2410,2411,1.850000,0.003540
3,2411,1618,1.420000,0.000156
4,2411,1617,2.190000,0.000648
...,...,...,...,...
10818,8724,8726,3.820000,0.000444
10819,7687,7686,0.469016,0.018577
10820,7877,7876,0.200000,0.021783
10821,8693,8692,0.836464,0.003790


In [9]:
df3.to_parquet(snakemake.output[0], index=False)